# Get CPI monthly details from BLS feed

#### Import Python tools

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import numpy as np
import altair as alt
import altair_stiles as altstiles
import requests
import json
from bs4 import BeautifulSoup

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [5]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [6]:
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

## XyXyXy

#### XyXyXy

In [27]:
url = "https://www.bls.gov/feed/cpi.rss"
headers = {
    "accept": "application/xml;q=0.9, */*;q=0.8",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}
response = requests.get(url, headers=headers)

In [167]:
df = (
    pd.read_xml(response.text, parse_dates=["published", "updated"])
    .drop(list(range(0, 8)))[
        [
            "title",
            "published",
            "content",
        ]
    ]
    .reset_index(drop=True)
)

In [168]:
df[["month", "nu_title", "sa", "annual", "rest"]] = df["content"].str.split(
    ",", expand=True
)

In [169]:
df["annual_pct_change"] = (
    df["annual"]
    .str.replace("and rose ", "")
    .str.replace(" percent over the last 12 months", "")
    .astype(float)
)

In [170]:
df["published"] = pd.to_datetime(df["published"]).dt.date
df["year"] = pd.to_datetime(df["published"]).dt.year

In [171]:
df[["rest", "annual_pct_change_less"]] = df["rest"].str.split(";", expand=True)

In [172]:
df["annual_pct_change_less"] = (
    df["annual_pct_change_less"]
    .str.replace("up ", "")
    .str.replace(" percent over the year (NSA).", "", regex=False)
    .astype(float)
)

In [173]:
df["month"] = df["month"].str.replace("In ", "")

In [234]:
df_slim = df[
    [
        "published",
        "year",
        "month",
        "annual_pct_change",
        "annual_pct_change_less",
    ]
].copy()

In [247]:
cond1 = df_slim["annual_pct_change"][0] < df_slim["annual_pct_change"][1]
cond2 = df_slim["annual_pct_change"][0] > df_slim["annual_pct_change"][1]
cond3 = df_slim["annual_pct_change"][0] == df_slim["annual_pct_change"][1]

In [250]:
np.select([cond1, cond3, cond3], ["certain", "possible", "probable"], default="Other")

array('certain', dtype='<U8')

In [252]:
def calc_category(row):
    if row.annual_pct_change[0] < row.annual_pct_change[1]:
        print("yes")
    else:
        print("no")

In [255]:
df_slim

,published,year,month,annual_pct_change,annual_pct_change_less,category
0,2023-12-12,2023,November,3.1,4.0,certain
1,2023-11-14,2023,October,3.2,4.0,certain
2,2023-10-12,2023,September,3.7,4.1,certain
3,2023-09-13,2023,August,3.7,4.3,certain
4,2023-08-10,2023,July,3.2,4.7,certain
5,2023-07-12,2023,June,3.0,4.8,certain
6,2023-06-13,2023,May,4.0,5.3,certain
7,2023-05-10,2023,April,4.9,5.5,certain
8,2023-04-12,2023,March,5.0,5.6,certain
9,2023-03-14,2023,February,6.0,5.5,certain


In [213]:
df_slim

,published,year,month,annual_pct_change,annual_pct_change_less
0,2023-12-12,2023,November,3.1,4.0
1,2023-11-14,2023,October,3.2,4.0
2,2023-10-12,2023,September,3.7,4.1
3,2023-09-13,2023,August,3.7,4.3
4,2023-08-10,2023,July,3.2,4.7
5,2023-07-12,2023,June,3.0,4.8
6,2023-06-13,2023,May,4.0,5.3
7,2023-05-10,2023,April,4.9,5.5
8,2023-04-12,2023,March,5.0,5.6
9,2023-03-14,2023,February,6.0,5.5


In [182]:
df_slim

,published,year,month,annual_pct_change,annual_pct_change_less
0,2023-12-12,2023,November,3.1,4.0
1,2023-11-14,2023,October,3.2,4.0
2,2023-10-12,2023,September,3.7,4.1
3,2023-09-13,2023,August,3.7,4.3
4,2023-08-10,2023,July,3.2,4.7
5,2023-07-12,2023,June,3.0,4.8
6,2023-06-13,2023,May,4.0,5.3
7,2023-05-10,2023,April,4.9,5.5
8,2023-04-12,2023,March,5.0,5.6
9,2023-03-14,2023,February,6.0,5.5


---

## XyXyXy

#### XyXyXy